In [1]:
!pip install -q datasets

In [2]:
from datasets import load_dataset
dataset = load_dataset("ailsntua/QEvasion")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

In [3]:
train_ds = dataset["train"]
test_ds = dataset["test"]

In [4]:
label2id = {
    "Clear Reply": 0,
    "Ambivalent": 1,
    "Clear Non-Reply": 2
}

id2label = {v:k for k,v in label2id.items()}


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [34]:
def preprocess_clarity(batch):

    texts = [
        str(q) + " " + str(a)
        for q, a in zip(batch["interview_question"], batch["interview_answer"])
    ]

    tokenized = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=512
    )

    tokenized["labels"] = [
        label2id[label]
        for label in batch["clarity_label"]
    ]

    return tokenized


In [7]:
train_enc = train_ds.map(
    preprocess_clarity,
    batched=True,
    remove_columns=train_ds.column_names
)

train_enc.set_format("torch")



Map:   0%|          | 0/3448 [00:00<?, ? examples/s]

In [8]:
test_enc = test_ds.map(
    preprocess_clarity,
    batched=True,
    remove_columns=test_ds.column_names
)

test_enc.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

In [29]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np


def compute_metrics(eval_pred):

    logits, labels = eval_pred

    preds = logits.argmax(axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        preds,
        average="macro"
    )

    accuracy = accuracy_score(labels, preds)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "macro_f1": f1,
    }


In [12]:
"""

from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model_clarity = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

training_args = TrainingArguments(
    output_dir="clarity_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
)

trainer_clarity = Trainer(
    model=model_clarity,
    args=training_args,
    train_dataset=train_enc,
    eval_dataset=test_enc,
)

trainer_clarity.train()
"""


'\n\nfrom transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments\n\nmodel_clarity = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=3)\n\ntraining_args = TrainingArguments(\n    output_dir="clarity_model",\n    eval_strategy="epoch",\n    learning_rate=2e-5,\n    per_device_train_batch_size=16,\n    per_device_eval_batch_size=16,\n    num_train_epochs=3,\n)\n\ntrainer_clarity = Trainer(\n    model=model_clarity,\n    args=training_args,\n    train_dataset=train_enc,\n    eval_dataset=test_enc,\n)\n\ntrainer_clarity.train()\n'

In [14]:
dataset_split = train_enc.train_test_split(test_size=0.2)

train_dataset = dataset_split["train"]
val_dataset = dataset_split["test"]


In [15]:
"""

trainer_clarity_v2 = Trainer(
    model=model_clarity,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer_clarity_v2.train()
"""

'\n\ntrainer_clarity_v2 = Trainer(\n    model=model_clarity,\n    args=training_args,\n    train_dataset=train_dataset,\n    eval_dataset=val_dataset,\n    compute_metrics=compute_metrics\n)\n\ntrainer_clarity_v2.train()\n'

In [37]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments


model_clarity = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=3)

training_args_v3 = TrainingArguments(
    output_dir="clarity_model",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=True,
    num_train_epochs=6,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_dir="./v3_logs",
)

trainer_clarity_v3 = Trainer(
    model=model_clarity,
    args=training_args_v3,
    train_dataset=train_enc,
    eval_dataset=test_enc,
    compute_metrics=compute_metrics
)

trainer_clarity_v3.train()

Loading weights:   0%|          | 0/389 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: roberta-large
Key                             | Status     | 
--------------------------------+------------+-
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
classifier.out_proj.bias        | MISSING    | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.dense.bias           | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.
`logging_dir` is deprecated

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Macro F1
1,No log,0.804607,0.685065,0.400235,0.372639,0.348418
2,No log,0.803352,0.665584,0.547859,0.553712,0.522447
3,0.876120,0.744257,0.681818,0.619083,0.582350,0.532169
4,0.876120,0.696946,0.714286,0.765977,0.537154,0.586596
5,0.602110,0.718120,0.685065,0.581527,0.538067,0.555620
6,0.602110,0.733147,0.668831,0.560538,0.563530,0.561941


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=1296, training_loss=0.675408587043668, metrics={'train_runtime': 1460.2779, 'train_samples_per_second': 14.167, 'train_steps_per_second': 0.888, 'total_flos': 9639930593452032.0, 'train_loss': 0.675408587043668, 'epoch': 6.0})

In [31]:
from sklearn.model_selection import train_test_split

train_indices, val_indices = train_test_split(
    np.arange(len(train_enc)),
    test_size=0.2,
    stratify=train_enc['labels'],
    random_state=42
)
train_dataset = train_enc.select(train_indices)
val_dataset = train_enc.select(val_indices)


In [32]:
from sklearn.metrics import f1_score
import numpy as np

predictions = trainer_clarity_v3.predict(test_enc)

preds = np.argmax(predictions.predictions, axis=1)

labels = predictions.label_ids

macro_f1 = f1_score(labels, preds, average="macro")

print("Macro F1:", macro_f1)


Macro F1: 0.5493738889152647


In [33]:
predicted_labels = [
    id2label[p]
    for p in preds
]

import pandas as pd

submission = pd.DataFrame({
    "clarity_prediction": predicted_labels
})

submission.to_csv("clarity_predictions_v2.csv", index=False)
